In [1]:
!git clone https://github.com/The-AI-Alliance/proscenium.git

fatal: destination path 'proscenium' already exists and is not an empty directory.


In [2]:
%cd proscenium

/content/proscenium


In [3]:
!python -m pip install .

Processing /content/proscenium
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached datasets-3.4.1-py3-none-any.whl.metadata (19 kB)
  Using cached eyecite-2.6.11-py3-none-any.whl.metadata (22 kB)
  Using cached gofannon-0.25.13-py3-none-any.whl.metadata (6.0 kB)
  Using cached langchain_huggingface-0.1.2-py3-none-any.whl.metadata (1.3 kB)
  Using cached neo4j-5.28.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached stringcase-1.2.0.tar.gz (3.0 kB)
  Preparing metadata (setup.py) ... done
  Using cached thespian-4.0.1.tar.gz (437 kB)
  Preparing metadata (setup.py) ... done
  Using cached tiktoken-0.9.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached anthropic-0.30.1-py3-none-any.whl.metadata (18 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.me

In [4]:
import os
from google.colab import userdata

api_key = userdata.get('TOGETHER_API_KEY')
os.environ['TOGETHER_API_KEY'] = api_key

In [5]:
from rich import print
from rich.panel import Panel
from rich.prompt import Prompt

import asyncio

import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [6]:
from demo.config import default_model_id

model_id = default_model_id
print(model_id)

together:meta-llama/Llama-3-70b-chat-hf

# Prepare Vector Database from Document Chunks

In [7]:
!pip install langchain_community

In [8]:
import nest_asyncio
nest_asyncio.apply()

In [9]:
import demo.domains.literature as literature_config
from proscenium.verbs.read import url_to_file

asyncio.run(url_to_file(literature_config.url, literature_config.data_file))

print("Data file to chunk:", literature_config.data_file)


Data file to chunk: four_plays_of_aeschylus.txt

In [10]:
!pip install pymilvus

In [11]:
!pip install pymilvus.model

In [12]:
from proscenium.verbs.vector_database import embedding_function

embedding_fn = embedding_function(literature_config.embedding_model_id)
print("Embedding model", literature_config.embedding_model_id)

Embedding model all-MiniLM-L6-v2

In [13]:
from proscenium.verbs.vector_database import create_vector_db

milvus_uri = "file:/milvus.db"

vector_db_client = create_vector_db(milvus_uri, embedding_fn, overwrite=True)
print("Vector db at uri", milvus_uri)

Vector db at uri file:/milvus.db

In [14]:
!pip install aisuite

In [15]:
from proscenium.scripts.rag import build_vector_db

build_vector_db(literature_config.data_file, vector_db_client, embedding_fn)

# vector_db_client.close()

Data file four_plays_of_aeschylus.txt has 268 chunks

268 chunks inserted

╭─────────────────────────────────────────────── Collection chunks ───────────────────────────────────────────────╮
│ ╭────────────────────────────────────────────────── Params ───────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ │     Collection Name: chunks                                                                                 │ │
│ │     Auto ID: True                                                                                           │ │
│ │     Num Shards: 0                                                                                           │ │
│ │     Description: Chunks Schema                                                                              │ │
│ │     Functions: []                                                                                           │ │
│ │     Aliases: []                                                                                             │ │
│ │     Collection ID: 0                                                                                        │ │
│ │     Consistency Level: 0                                                                                    │ │
│ │     Properties: {}                                                                                          │ │
│ │     Num Partitions: 0                                                                                       │ │
│ │     Enable Dynamic Field: True                                                                              │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│                                          Fields                                                                 │
│ ┏━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┓                        │
│ ┃ id  ┃ name   ┃ description ┃ type         ┃ params            ┃ auto_id ┃ is_primary ┃                        │
│ ┡━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━┩                        │
│ │ 100 │ id     │             │ INT64        │                   │ True    │ True       │                        │
│ ├─────┼────────┼─────────────┼──────────────┼───────────────────┼─────────┼────────────┤                        │
│ │ 101 │ text   │             │ VARCHAR      │ max_length: 50000 │ -       │ -          │                        │
│ ├─────┼────────┼─────────────┼──────────────┼───────────────────┼─────────┼────────────┤                        │
│ │ 102 │ vector │             │ FLOAT_VECTOR │ dim: 384          │ -       │ -          │                        │
│ └─────┴────────┴─────────────┴──────────────┴───────────────────┴─────────┴────────────┘                        │
│ ╭─────────────────────────────────────────────────── Stats ───────────────────────────────────────────────────╮ │
│ │ row_count: 268                                                                                              │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Answer User Question

In [16]:
question = "What did Hermes say to Prometheus about giving fire to humans?"

In [17]:
#embedding_fn = embedding_function(literature_config.embedding_model_id)
#print("Embedding model:", literature_config.embedding_model_id)

#vector_db_client = vector_db(milvus_uri)
#print("Vector db at uri", milvus_uri)


In [20]:
from proscenium.scripts.rag import answer_question

answer = answer_question(question, literature_config.model_id, vector_db_client, embedding_fn)

print(Panel(answer, title="Assistant"))

╭───────────────────────────────────────────────────── User ──────────────────────────────────────────────────────╮
│ What did Hermes say to Prometheus about giving fire to humans?                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Found 4 closest chunks

                                            Closest Chunks                                            
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 id ┃ distance           ┃                                              entity.text ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 456885564141994220 │ 0.6078414916992188 │                                              PROMETHEUS. │
│                    │                    │              Proud? in such pride then be my foemen set, │
│                    │                    │                  And I to see—and of such foes art thou! │
│                    │                    │                                                          │
│                    │                    │                                                  HERMES. │
│                    │                    │            What, blam’st thou me too for thy sufferings? │
│                    │                    │                                                          │
│                    │                    │                                              PROMETHEUS. │
│                    │                    │            Mark a plain word—I loathe all gods that are, │
│                    │                    │             Who reaped my kindness and repay with wrong. │
│                    │                    │                                                          │
│                    │                    │                                                  HERMES. │
│                    │                    │                   I hear no little madness in thy words. │
│                    │                    │                                                          │
│                    │                    │                                              PROMETHEUS. │
│                    │                    │                Madness be mine, if scorn of foes be mad. │
│                    │                    │                                                          │
│                    │                    │                                                  HERMES. │
│                    │                    │               Past bearing were thy pride, in happiness. │
│                    │                    │                                                          │
│                    │                    │                                              PROMETHEUS. │
│                    │                    │                                                   Ah me! │
│                    │                    │                                                          │
│                    │                    │                                                  HERMES. │
│                    │                    │                     Zeus knoweth nought of sorrow’s cry! │
│                    │                    │                                                          │
│                    │                    │                                              PROMETHEUS. │
│                    │                    │        He shall! Time’s lapse bringeth all lessons home. │
│                    │                    │                                                          │
│                    │                    │                                                  HERMES. │
│                    │                    │             To thee it brings not yet discretion’s curb. │
│                    │                    │                                                          │
│                    │                    │                                              PROMETHEUS. │
│                    │                    │                 No—else I had not wrangled with a slave! │
│                    │                    │                                           

RAG prompt created. Calling inference at together:meta-llama/Llama-3-70b-chat-hf

╭─────────────────────────────────────────────────── Assistant ───────────────────────────────────────────────────╮
│ Hermes did not say anything to Prometheus about giving fire to humans in the provided chunks. The information   │
│ about Prometheus giving fire to humans is mentioned in Chunk 456885564141994203, but it is Prometheus who says  │
│ it to Io, not Hermes.                                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯